In [1]:
import warnings
warnings.filterwarnings("ignore", message="networkx backend defined more than once")
import sys
import gc
import os
import csv
import copy
import json
import math
import shutil
import random
import pickle
import itertools
import numpy as np
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
from protocols import MPC_protocol, MPG_protocol, SP_protocol
from graph import network, set_p_edge

from joblib import Parallel, delayed
from itertools import product
from networkx.algorithms.community import greedy_modularity_communities
from networkx.drawing.layout import *

# 获取当前 Notebook 的绝对路径
notebook_path = os.path.abspath("")
from config import DATA_PATHS



In [ ]:
mkr = ['x','+','d','o','1','2']+['x']*100
dashs = ['-.','--',':','-']+['-']*100
cols = ['gray','g','b','orange','r','k','purple']+['k']*100
linewidth = 2.2
mks = 5.5
fontsize = 14
sys.path.append("..")  # 确保根目录在 Python 路径中
root_path = DATA_PATHS["input_graphs"]
# LOOP_STATE_PATH = "loop_state.pkl"
# TEMP_DATA_PATH = "temp_data.pkl"

Find the ER for the MPC, MPG, and SP protocols

In [3]:
def load_data(filepath):
    pos = {}
    user = []

    # Step 1: 读取 JSON 文件
    with open(filepath, "r") as f:
        data = json.load(f)

    # Step 2: 初始化图
    G = nx.Graph()

    # Step 3: 添加节点

    for node in data["nodes"]:
        node_id = node["id"]
        x, y = node["latitude"], node["longitude"]
        G.add_node(node_id, location=node["location"], country=node["country"])  # 添加节点到图
        pos[node_id] = (y, x)  # 保存节点位置，注意 (longitude, latitude)

    # Step 4: 添加边
    for edge in data["links"]:
        source = int(edge["source"])
        target = int(edge["target"])
        G.add_edge(source, target, length=edge["length"])  # 添加边到图

    degree_dict = dict(G.degree())
    degree_items = list(degree_dict.items())
    first_node,first_degree = degree_items[0]
    print(f"First node ID: {first_node}, Degree: {first_degree}")

    user.append(data["nodes"][0]["id"])

    return G,user,pos

In [4]:
def multi_iterative_score_partition_with_drawing(
    G, 
    fixed_node, 
    alpha=1.0, 
    beta=1.0,
    max_rounds=10,
    shuffle_nodes=True,
    pos=None,
    output_path=None,
    sample_size=100  # 新增参数，默认只取 100 组组合
):
    """
    多轮迭代版“打分 + 搬家”分社区并筛选出 100 个组合：
      - 社区数 = degree(fixed_node) + 1
      - 初始: 0 号社区放 fixed_node，其余邻居各占一社区
      - 分配时: score = alpha * distance + beta * community_size
      - 多轮: 若搬家能让节点的 score 更低，则搬家，直至收敛或达到 max_rounds
      - 只随机返回 100 个组合，以防止组合数过多导致 MemoryError

    返回：
      - 最终的社区划分 communities
      - 100 个 key_nodes 组合列表
    """
    
    # ============ 1) 初始化社区容器 ============
    neighbors = list(G.neighbors(fixed_node))
    num_communities = len(neighbors)  # 除 0 号社区外的社区数
    communities = [set() for _ in range(num_communities + 1)]
    visited = set()

    # 第 0 号社区放 fixed_node
    communities[0].add(fixed_node)
    visited.add(fixed_node)

    # 其余邻居各占一社区
    for i, nb in enumerate(neighbors, start=1):
        communities[i].add(nb)
        visited.add(nb)

    # ============ 2) 初次分配剩余节点 ============
    for node in G.nodes():
        if node not in visited:
            best_score = float('inf')
            best_index = None
            # 尝试放入各社区，选择 score 最低者
            for i, nb in enumerate(neighbors, start=1):
                dist = nx.shortest_path_length(G, source=node, target=nb)
                size = len(communities[i])
                score = alpha * dist + beta * size
                if score < best_score:
                    best_score = score
                    best_index = i
            communities[best_index].add(node)
            visited.add(node)

    # ============ 3) 多轮迭代搬家 ============
    round_num = 0
    while round_num < max_rounds:
        round_num += 1
        moved_count = 0

        # 获取除 fixed_node 外的所有节点
        all_nodes = [n for n in G.nodes() if n != fixed_node]

        if shuffle_nodes:
            random.shuffle(all_nodes)

        for node in all_nodes:
            # 找到当前所在社区
            current_idx = None
            for i, comm in enumerate(communities):
                if node in comm:
                    current_idx = i
                    break
            
            # 固定节点不搬家
            if current_idx == 0:
                continue

            # 当前社区 score
            nb_current = neighbors[current_idx - 1]  # 对应邻居
            dist_current = nx.shortest_path_length(G, source=node, target=nb_current)
            size_current = len(communities[current_idx])
            current_score = alpha * dist_current + beta * size_current

            # 尝试搬家到其他社区
            best_score = current_score
            best_index = current_idx

            for i, nb in enumerate(neighbors, start=1):
                if i == current_idx:
                    continue
                dist = nx.shortest_path_length(G, source=node, target=nb)
                size = len(communities[i])
                score = alpha * dist + beta * size

                if score < best_score:
                    best_score = score
                    best_index = i

            # 若找到更优的社区，则搬家
            if best_index != current_idx:
                communities[current_idx].remove(node)
                communities[best_index].add(node)
                moved_count += 1

        # 若无人搬家，则收敛，退出迭代
        if moved_count == 0:
            break

    # ============ 4) 生成 key_nodes 组合 ============
    # 注意：0 号社区仅包含 fixed_node，不参与组合枚举
    all_key_nodes_combos = []
    if all(len(communities[i]) > 0 for i in range(1, num_communities + 1)):
        # 计算所有组合总数
        total_combos = 1
        for i in range(1, num_communities + 1):
            total_combos *= len(communities[i])

        # 随机采样 100 个组合（如果组合数少于 100，则返回所有组合）
        if total_combos <= sample_size:
            all_products = product(*(communities[i] for i in range(1, num_communities + 1)))
            for combo in all_products:
                combo_list = [fixed_node] + list(combo)
                all_key_nodes_combos.append(combo_list)
        else:
            # 随机选择 100 个组合
            sampled_set = set()
            while len(all_key_nodes_combos) < sample_size:
                combo = tuple(random.choice(tuple(communities[i])) for i in range(1, num_communities + 1))
                if combo not in sampled_set:
                    sampled_set.add(combo)
                    all_key_nodes_combos.append([fixed_node] + list(combo))
    else:
        all_key_nodes_combos = []

    return communities, all_key_nodes_combos


In [5]:

def plot_er_vs_p(p_range, ER, funcs, cols, output_path, fontsize=12, figsize=(10, 6), dpi=600):
    """
    绘制 ER 与链接生成概率 p 的关系图，并保存为文件。

    参数:
        p_range (array-like): p 的取值范围。
        ER (list of lists): 每个函数对应的 ER 数据。
        funcs (list): 包含函数的列表，用于生成图例名称。
        cols (list): 每个函数对应的颜色列表。
        output_path (str): 保存图片的路径。
        fontsize (int, 可选): 坐标轴和标签字体大小，默认 12。
        figsize (tuple, 可选): 图形大小，默认 (10, 6)。
        dpi (int, 可选): 图像分辨率，默认 600。
    """
    nom_list = [str(f).split(' ')[1] for f in funcs]
    plt.figure(figsize =(10,6),dpi=600 )
    plt.grid(linewidth=0.5)
    for i in range(len(funcs)):
        y = plt.plot(p_range, ER[i],
                    color = cols[i],
                    marker = "x",
                    linestyle='None',
                    markersize = 3,
                    #  alpha = 0.5,
                    #  linewidth=linewidth,
                    label = nom_list[i])
    plt.yscale('log')
    plt.legend(fontsize=10)
    plt.tick_params(labelsize=fontsize)

    plt.xlabel('Link generation probability p',fontsize=fontsize)
    plt.ylabel('ER ($\mathregular{GHZ}_5/\ \\mathregular{T_{slot}}$)',fontsize=fontsize)

    ax = plt.gca()
    ax.set_xlim([0.2, 1])
    ax.set_ylim([0.0001, 1])
    # 保存图片
    plt.savefig(output_path, dpi=dpi)
    plt.close('all')
    print(f"Plot saved to {output_path}")

<>:32: SyntaxWarning: invalid escape sequence '\m'
<>:32: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Aaron\AppData\Local\Temp\ipykernel_17116\110213519.py:32: SyntaxWarning: invalid escape sequence '\m'
  plt.ylabel('ER ($\mathregular{GHZ}_5/\ \\mathregular{T_{slot}}$)',fontsize=fontsize)


In [6]:
def process_single_p(G, combo, p, funcs, timesteps, reps):
    """处理单个p值的并行任务函数"""
    local_G = G.copy()
    # 设置当前p值的边概率
    set_p_edge(local_G, p_op=p)
    
    # 为所有协议计算当前p的ER值
    p_ers = []
    for func in funcs:
        er, _, _ = func(local_G, combo, timesteps=timesteps, reps=reps)
        p_ers.append(er)
    
    return p_ers

In [ ]:
# ============== 2) 全局参数 ==============
funcs = [MPC_protocol, MPG_protocol, SP_protocol]
p_range = np.linspace(1, 0.2, 50)

timesteps = 100
reps = 200
alpha = 1.4
beta = 0.105
max_rounds = 10
shuffle_nodes = True

sr_results = []  # 每个文件的 SR 结果都会追加到这里

# 分段退出：针对“组合”数量
chunk_size = 300 
state_file = "loop_state1.pkl"

# ============== 3) 读取/初始化进度 ==============
try:
    with open(state_file, "rb") as f:
        progress = pickle.load(f)
    print("恢复进度：", progress)
except FileNotFoundError:
    progress = {
        "subfolder_idx": 0,   # 当前处理到第几个子文件夹
        "file_idx": 0,        # 当前子文件夹内处理到第几个文件
        "combo_idx": 0,       # 当前文件内处理到第几个组合
        "global_combo_count": 0  # 全局已处理的组合数
    }
    print("未发现进度文件，从头开始。")

# 子文件夹列表  
subfolders = [sf for sf in root_path.iterdir() if sf.is_dir()]
subfolders.sort()

# ============== 4) 主循环 ==============
for s_idx in range(progress["subfolder_idx"], len(subfolders)):
    subfolder = subfolders[s_idx]
    if not subfolder.is_dir():
        continue

    print(f"\nProcessing subfolder: {subfolder} ({s_idx+1}/{len(subfolders)})")

    # 遍历文件
    files = [f for f in subfolder.iterdir() if f.is_file()]
    files.sort()
    for f_idx in range(progress["file_idx"], len(files)):
        file = files[f_idx]
        if not file.is_file():
            continue

        print(f"  Processing file: {file} ({f_idx+1}/{len(files)}) ...")

        # 每个文件：初始化 counters
        failure_counts = {func.__name__: 0 for func in funcs}
        combination_counter = 0

        # 加载图
        G, users, pos = load_data(file)
        # nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray")
        # plt.close()

        G = network(G)

        # 输出路径
        class_folder = subfolder.name
        file_path = file.with_suffix(".png")
        file_name = file.stem

        er_folder_path = Path.cwd().parent.joinpath("new_result1", class_folder)
        er_folder_path.mkdir(exist_ok=True)
        er_topology_folder_path = er_folder_path.joinpath(file_name)
        er_topology_folder_path.mkdir(exist_ok=True)
        communities_output_path = Path.cwd().parent.joinpath("communitie", class_folder, file_name)

        communities, users_node_combination = multi_iterative_score_partition_with_drawing(
            G, users[0], alpha, beta, max_rounds, shuffle_nodes, pos, communities_output_path
        )

        # 采样组合，这里从所有组合中随机采样 100 个
        sampled_combinations = np.random.choice(
            len(users_node_combination),
            size=min(100, len(users_node_combination)),
            replace=False
        )

        # 用 tqdm 显示处理组合的进度
        for sampled_idx, original_idx in enumerate(tqdm(sampled_combinations, desc="  Processing combinations", unit="combo")):
            combo = users_node_combination[original_idx]
            combination_counter += 1
            progress["global_combo_count"] += 1

            # =========== 新增：组合详细信息 ===========
            combination_sr = {
                "Combination_ID": f"combo_{sampled_idx}",
                "Nodes": str(combo),  # 将节点列表转为字符串，如 "[2,5,9]"
            }

            # 计算 ER
            ER = np.zeros((len(funcs), len(p_range)))
            results = Parallel(n_jobs=-1, verbose=10)(
                delayed(process_single_p)(G, combo, p, funcs, timesteps, reps)
                for p in p_range
            )

            # 将结果填充到 ER 矩阵
            for i, p_ers in enumerate(results):
                ER[:, i] = p_ers

            plot_er_vs_p(p_range, ER, funcs, cols, er_topology_folder_path.joinpath(f'result_for_{str(combo)}'))

            # =========== 计算成功率比例 ===========
            for func_idx, func in enumerate(funcs):
                protocol_er = ER[func_idx, :]
                zero_count = np.sum(protocol_er < 1e-10)
                success_ratio = 1 - (zero_count / len(p_range))  # 计算成功率比例
                combination_sr[func.__name__] = round(success_ratio, 3)  # 保留3位小数

            del results, ER
            gc.collect()

            # =========== 写入CSV文件 ===========
            output_subfolder_csv_path = er_topology_folder_path.joinpath(f"{file.stem}_sr_details.csv")

            # 定义字段（首次运行时写入表头）
            fieldnames = ["Combination_ID", "Nodes"] + [func.__name__ for func in funcs]

            # 判断是否需要写表头
            write_header = not output_subfolder_csv_path.exists()

            with open(output_subfolder_csv_path, mode="a", newline="") as subfile:
                csv_writer = csv.DictWriter(subfile, fieldnames=fieldnames)
                if write_header:
                    csv_writer.writeheader()
                csv_writer.writerow(combination_sr)

            # 分段退出：每处理 chunk_size 个组合后退出
            if progress["global_combo_count"] % chunk_size == 0:
                print(f"\n已处理 {progress['global_combo_count']} 个组合，准备退出。")
                progress["subfolder_idx"] = s_idx
                progress["file_idx"] = f_idx
                progress["combo_idx"] = sampled_idx + 1  # 下次从下一个组合继续
                with open(state_file, "wb") as pf:
                    pickle.dump(progress, pf)
                exit()

            # 每个组合处理完更新进度
            progress["combo_idx"] = sampled_idx + 1
            progress["subfolder_idx"] = s_idx
            progress["file_idx"] = f_idx
            with open(state_file, "wb") as pf:
                pickle.dump(progress, pf)


        # 这个文件处理完 => 重置 combo_idx，并 file_idx+1
        progress["combo_idx"] = 0
        progress["file_idx"] = f_idx + 1
        with open(state_file, "wb") as pf:
            pickle.dump(progress, pf)


恢复进度： {'subfolder_idx': 0, 'file_idx': 12, 'combo_idx': 39, 'global_combo_count': 1103}

Processing subfolder: F:\Third-year\Third-year project\quantum_repeaters_testing\graphs_json\class_0 (1/3)
  Processing file: F:\Third-year\Third-year project\quantum_repeaters_testing\graphs_json\class_0\TOP_41_GTSPOLAND.json (13/36) ...
First node ID: 3, Degree: 2


  Processing combinations:   0%|          | 0/100 [00:00<?, ?combo/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  17 out of  50 | elapsed:    2.6s remaining:    5.1s
[Parallel(n_jobs=-1)]: Done  23 out of  50 | elapsed:    4.8s remaining:    5.6s
[Parallel(n_jobs=-1)]: Done  29 out of  50 | elapsed:    6.4s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done  35 out of  50 | elapsed:    9.6s remaining:    4.0s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   11.1s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   12.1s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   12.5s finished
  Processing combinations:   1%|          | 1/100 [00:13<22:17, 13.51s/combo]

Plot saved to f:\Third-year\Third-year project\quantum_repeaters_testing\new_result1\class_0\TOP_41_GTSPOLAND\result_for_[3, 25, 7]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  17 out of  50 | elapsed:    2.0s remaining:    4.0s
[Parallel(n_jobs=-1)]: Done  23 out of  50 | elapsed:    4.0s remaining:    4.7s
[Parallel(n_jobs=-1)]: Done  29 out of  50 | elapsed:    7.1s remaining:    5.1s
[Parallel(n_jobs=-1)]: Done  35 out of  50 | elapsed:    8.7s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   10.0s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   11.1s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   11.7s finished
  Processing combinations:   2%|▏         | 2/100 [00:26<21:12, 12.99s/combo]

Plot saved to f:\Third-year\Third-year project\quantum_repeaters_testing\new_result1\class_0\TOP_41_GTSPOLAND\result_for_[3, 25, 17]


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  17 out of  50 | elapsed:  1.0min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  23 out of  50 | elapsed:  1.6min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  29 out of  50 | elapsed:  1.7min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  35 out of  50 | elapsed:  1.8min remaining:   45.9s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:  1.9min remaining:   25.1s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  2.0min remaining:    7.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.0min finished
  Processing combinations:  64%|██████▍   | 64/100 [1:25:55<1:02:22, 103.95s/combo]

Plot saved to f:\Third-year\Third-year project\quantum_repeaters_testing\new_result1\class_0\TOP_43_HIBERNIAGLOBAL\result_for_[40, 19, 27, 44, 36]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   58.4s
[Parallel(n_jobs=-1)]: Done  17 out of  50 | elapsed:  1.1min remaining:  2.1min
  Processing combinations:  64%|██████▍   | 64/100 [1:27:51<49:25, 82.37s/combo]   


KeyboardInterrupt: 

: 